<h1>Using the total derivative in a simple neural network</h1>

The purpose of this notebook is to clarify the computational mechanics of a simple neural network by <i>using the matrix version of the chain rule.</i>

This network's dummy task is to learn a random noise under the guise of a single valued function of two variables 

$$f(x_1,x_2)=y.$$

To achieve this, it uses a single three dimensional hidden layer. Mathematically, we are looking at the following vector spaces and maps:

$$\mathbb{R}^2\oplus\mathbb{1}\xrightarrow{w^1}\mathbb{R}^3\xrightarrow{\sigma}\mathbb{R}^3\oplus\mathbb{1}\xrightarrow{w^2}\mathbb{R}.$$

In this sequence, $w^1$ & $w^2$ are linear transformations, $\sigma$ is our non-linear activation function  and the $\mathbb{1}$'s correspond to our bias terms. 

Concretely, $\mathbb{R}^n\oplus\mathbb{1}$ consists of vectors in $\mathbb{R}^{n+1}$ with last coordinate equal to $1$. This lets us keep track of bias using a linear transformation at each step.

To begin training our network, we randomly initialize the linear transformations $w^1$ and $w^2$ and we walk through a forward pass.

In [121]:
import numpy as np

# set dimensions
N_in, N_hid, N_out = 2, 3, 1

# generate random input-output
x = np.random.randint(10, size=(2, 1))
y = np.random.randint(10, size=(1, 1))

# randomly initialize our weights and biases
w1 = np.random.randint(10, size=(N_hid, N_in + 1))
w2 = np.random.randint(10, size=(N_out, N_hid + 1))
print("Here's w1:\n", w1, "\nHere's w2:\n", w2)


Here's w1:
 [[1 5 5]
 [9 9 9]
 [2 5 9]] 
Here's w2:
 [[6 6 7 5]]


Now that we have initialized $w^1$ and $w^2$ as random linear transformations, our goal is to progressively improve them. 

To do this, we first have to choose an error function with respect to which we want to optimize. Let's go with the usual Euclidean distance.

We can now compare the output of our network with the expected output.

In [126]:
# define our activation function ReLU
def sigma(z):
    return np.maximum(z, 0)


# forward pass
z0 = np.append(x, [1])
a0 = z0.reshape(3, 1)
z1 = w1 @ (a0)
a1 = np.vstack((sigma(z1), [1]))
a2 = w2 @ (a1)

# compute and print current loss
firstloss = float(np.square(a2 - y))
print("Here's our first loss:", firstloss)

Here's our first loss: 576313.9241507892


Not even close... which is not much of a surprise since everything in sight was random.

<h2> The dual point of view </h2>

To improve our network, we shift our perspective on the sequence of vector spaces above by  viewing the entries of the matrices $w^1$ and $w^2$ as variables to be optimized. To do this precisely, let's first label the variables  in our diagram as follows:

\begin{equation}\tag{1}
\mathbb{R}^2\oplus\mathbb{1}\xrightarrow{w^1}\mathbb{R}^3\xrightarrow{\sigma}\mathbb{R}^3\oplus\mathbb{1}\xrightarrow{w^2}\mathbb{R}.
\end{equation}


$$
 a^0\,\,\,\mapsto\,\,\, \,z^1\,\,\,\,\mapsto \,\,\,\,a^1\,\,\,\mapsto\,\,\, a^2.
$$

In other words, set $z^1:= w^1(a^0)$, $a^1:=\sigma(z^1)\oplus 1$ and $a^2:=w^2(a^1)$ as in the forward pass code above.

The key point is that <i>we can now use this data to think about $w^1$ & $w^2$ as vectors and $a^1$ & $a^2$ as  linear transformations</i> in the following "dualised" diagram:

\begin{equation}\tag{2}
\mathbb{R}^9\,\xrightarrow{\alpha^0}\,\mathbb{R}^3\,\xrightarrow{\sigma}\,\mathbb{R}^4\,\xrightarrow{\alpha^1}\,\mathbb{R}
\end{equation}
$$
w^1\,\mapsto \,\,z^1\,\,\mapsto\,\,w^2\,\,\mapsto\,\, a^2.
$$

Here, $\alpha^0(w^1):=w^1(a^0)$ and $\alpha^1(w^2):=w^2(a^1)$ are evaluation maps; the copy of $\mathbb{R}^9$ corresponds to the nine weights and biases of

$$w^1=\begin{pmatrix}
w^1_{1,1} & w^1_{1,2} & b^1_1\\
w^1_{2,1} & w^1_{2,2} & b^1_2\\
w^1_{3,1} & w^1_{3,2} & b^1_3
\end{pmatrix}$$

and the second copy of $\mathbb{R}^4$ corresponds to the four weights and biases of  

$$w^2=\left(w^2_{1,1}, w^2_{1,2}, w^2_{1,3}, b^2_1\right).$$

Since we will need to work explicitly with $\alpha^0$ and $\alpha^1$, let's find their matrix representations in the "row-by-row" bases

$$\{w^1_{1,1},w^1_{1,2},b^1_1,w^1_{2,1},w^1_{2,2},b^1_2,w^1_{3,1},w^1_{3,2},b^1_3\}\text{ for }\mathbb{R}^6\text{ and }\{w^2_{1,1},w^2_{1,2},w^2_{1,3},b^2_1\}\text{ for }\mathbb{R}^3.$$

In these coordinates, $\alpha^0$ is given by the matrix $ I_3 \otimes (a^0)^T$ which is just a fancy way to write the "row diagonal" matrix with repeated copies of $(a^0)^T$. Similarily, $\alpha^1$ is given by the matrix $(a^1)^T=I_1\otimes (a^1)^T$. Here, as usual, $I_k$ denotes the $k\times k$ identity matrix. 

<h2> Computing gradients </h2>

We can now compare the output of the network to the expected output and compute the gradient of the error with respect to the $w^i_{j,k}$ and $b^i_j$'s.

Explicitly, we extend $(2)$ to include our error function

$$\tag{3}
\mathbb{R}^9\xrightarrow{\alpha^0}\mathbb{R}^3\xrightarrow{\sigma}\mathbb{R}^4\xrightarrow{\alpha^1}\mathbb{R}\xrightarrow{J}\mathbb{R}
$$
$$
w^1\,\mapsto \,\,z^1\,\,\mapsto\,\,w^2\,\,\mapsto\,\, a^2\,\,\mapsto e
$$

where $J(a^2):=(a^2-y)^2$. Our goal is to compute the gradient of $J$ with respect to the variables $w^i_{j,k}$ and $b^i_j$. To do this, we compute all partial derivatives $$\frac{\partial \,J}{\,\,\partial \, w^i_{j,k}}\text{ and }\frac{\partial\, J}{\,\,\partial \,b^i_{j}}$$ using the multi-variable chain rule and working our way backwards through the layers of the neural network.



Let $D_x(F)$ denote the total derivative (Jacobian) of a function $F:\mathbb{R}^n\rightarrow\mathbb{R}^m$ evaluated at $x\in\mathbb{R}^n$. Starting at the last layer of our network, we have

$$D_{w^2}(J \circ\alpha^1)=D_{a^2}(J)\circ D_{w^2}(\alpha^1)$$

where  composition corresponds to matrix multiplication. The first term is easy to compute: $D_{a^2}(J)=2(a^2-y).$ For the second term, we use the fact that a linear transformation is its own derivative to get that $D_{w^2}(\alpha^1) = (a^1)^T.$ Combining them produces a vector of partial derivatives coming from $w^2$:

$$\tag{4}D_{w^2}(J \circ\alpha^1)=\left(
\frac{\partial J\,}{\partial w^2_{1,1}}, \frac{\partial J\,}{\partial w^2_{1,2}}, \frac{\partial J\,}{\partial w^2_{1,3}},  \frac{\partial J\,}{\partial b^2_1}\right).$$

In [123]:
D_a2 = 2.0 * (a2 - y)
D_w2 = (a1.T)

grad_w2 = D_a2 @ D_w2
print("Here are the partial derivatives coming from w2:\n", grad_w2)


Here are the partial derivatives coming from w2:
 [[ 38544. 115632.  51392.   1606.]]


We now iterate further back down the layers of $(3)$

$$
\mathbb{R}^9\xrightarrow{\alpha^0}\mathbb{R}^3\xrightarrow{\sigma}\mathbb{R}^4\xrightarrow{\alpha^1}\mathbb{R}\xrightarrow{J}\mathbb{R}
$$

 to find the partials coming from $w^1$. To clarify our steps, let $F:=J\circ \alpha^1$ and $G:=\sigma\circ \alpha^0$. By the chain rule:

$$D_{w^1}(F\circ G)=D_{G(w^1)}(F)\circ D_{w^1}(G)=D_{w^2}(J\circ \alpha^1)\circ D_{w^1}(\sigma\circ \alpha^0).$$

Notice that we've already computed the first term! The <i>backpropagation algorithm</i> capitalizes on this observation to implement an efficient computation of all required partial derivatives. Moving along, we  compute the remaining term  

$$D_{w^1}(\sigma\circ\alpha^0)=D_{z^1}(\sigma)\circ D_{w^1}(\alpha^0).$$

The first term $D_{z^1}(\sigma)$ is a bit subtle.  To be precise, we interpret the map $\sigma:\mathbb{R}^3\rightarrow\mathbb{R}^4$ as the function $(u,v,w)\mapsto (\max(u,0), \max(v,0), \max(w,0),1)$. Its derivative is therefore a diagonal $3\times 3$ matrix with zeroes wherever $z^1$ is negative and ones wheverer $z^1$ is positive along with a $4$th row consisting of three zeroes. For the second term, as before, $D_{w^1}(\alpha^0)=\left[I_3\otimes (a^0)^T\right]$ since a linear transformation is its own derivative.

Putting this all together produces a vector of partial derivatives

$$\tag{5}
D_{w^1}(J\circ \alpha^1\circ\sigma\circ \alpha^0 )= \left(\frac{\partial J\,}{\partial w^1_{1,1}}, \frac{\partial J\,}{\partial w^1_{1,2}}, \frac{\partial J\,}{\partial b^1_1}, 
\frac{\partial J\,}{\partial w^1_{2,1}}, \frac{\partial J\,}{\partial w^1_{2,2}},  \frac{\partial J\,}{\partial b^1_2}, 
\frac{\partial J\,}{\partial w^1_{3,1}},  \frac{\partial J\,}{\partial w^1_{3,2}}, \frac{\partial J\,}{\partial b^1_1} \right).
$$



In [127]:
D_z1 = z1.copy()
D_z1[z1 < 0] = 0
D_z1[z1 > 0] = 1
D_z1 = D_z1.T * np.identity(3)
D_z1 = np.vstack((D_z1, [0, 0, 0]))

D_w1 = np.kron(np.identity(3), (a0.T))

grad_w1 = grad_w2 @ D_z1 @ D_w1

print("Here are the partial derivatives coming from w1:\n", grad_w1)


Here are the partial derivatives coming from w1:
 [[154176. 115632.  38544. 462528. 346896. 115632. 205568. 154176.  51392.]]


We are now ready to nudge our linear transformations $w^1$ and $w^2$ along the negative of the gradients computed in $(4)$ and $(5)$ to see if we improved.

In [128]:
# set our learning rate
eta = 1e-6

# update our weights
w2 = w2 - eta * grad_w2
w1 = w1 - eta * np.reshape(grad_w1, (3, 3))

# next forward pass
z0 = np.append(x, [1])
a0 = z0.reshape(3, 1)
z1 = w1 @ (a0)
a1 = np.vstack((sigma(z1), [1]))
a2 = w2 @ (a1)

# compute and print error improvement
secondloss = float(np.square(a2 - y))
improvement = firstloss - secondloss
print("Our error has improved by:", improvement)

Our error has improved by: 63347.528627651336
